## INITIALIZE

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import importlib as il
from hypso import Hypso1, Hypso2
import src.deh as deh
il.reload(deh)

HYPSO_HEIGHT    =1092
HYPSO_WIDTH     =598
HYPSO_BANDS     =112 
wavelengths = [387.8475,  391.40405, 394.9594,  398.51355, 402.06647, 405.61816, 409.1686,
 412.71786, 416.2659,  419.8127,  423.3583,  426.90268, 430.44586, 433.9878,
 437.5285,  441.068,   444.6063,  448.14334, 451.67917, 455.2138,  458.7472,
 462.2794,  465.81033, 469.3401,  472.86862, 476.3959,  479.922,   483.44687,
 486.97052, 490.49295, 494.01416, 497.53415, 501.05292, 504.57047, 508.0868,
 511.6019,  515.1158,  518.6285,  522.1399,  525.65015, 529.1592,  532.667,
 536.1735,  539.6789,  543.18304, 546.686,   550.1876,  553.6881,  557.1874,
 560.6854,  564.18225, 567.67786, 571.17224, 574.6654,  578.15735, 581.6481,
 585.1376,  588.62585, 592.1129,  595.59875, 599.0834,  602.5668,  606.049,
 609.52997, 613.0097,  616.4882,  619.9656,  623.44165, 626.9165,  630.39014,
 633.8626,  637.3338,  640.80383, 644.2726,  647.7401,  651.2065,  654.6716,
 658.1355,  661.59814, 665.05963, 668.5199,  671.9789,  675.4367,  678.89325,
 682.34863, 685.8028,  689.25574, 692.7074,  696.1579,  699.6072,  703.05524,
 706.502,   709.94763, 713.392,   716.8352,  720.27716, 723.7179,  727.1574,
 730.5957,  734.0328,  737.4686,  740.90326, 744.3367,  747.76886, 751.1998,
 754.6296,  758.0581,  761.4855,  764.91156, 768.3364,  771.7601,  775.1825,
 778.60376, 782.02374, 785.4425,  788.8601,  792.2764,  795.6915,  799.10547,
 802.5181 ]

In [ ]:
HYPSO_1_NC_DIR  = r"D:\HYPSO_1_NC"
HYPSO_2_NC_DIR  = r"D:\HYPSO_2_NC"
DATA_DIR        = r"D:\Hierarchical Unmixing Label\hUH\data"
IMAGES_DIR      = DATA_DIR + r"\images"
DEH_DIR         = DATA_DIR + r"\deh_models"
LABELS_DIR      = DATA_DIR + r"\labels"
TREE_DIR        = DATA_DIR + r"\tree_models"
PRED_DIR        = DATA_DIR + r"\predictions"
BINARY_PRED_DIR = DATA_DIR + r"\binary_predictions"
DECODED_DIR     = DATA_DIR + r"\decoded_predictions"
# PIPELINE_DIR    = r"\\wsl.localhost\Ubuntu-24.04\home\lofty\CODE\onboard-pipeline-modules"
# BINARY_PRED_DIR = PIPELINE_DIR + r"\onboard_classification\onboard_classification\src\data\output"
# DECODED_DIR     = PIPELINE_DIR + r"\decoding\decoded_output"
hUH_model_file='10img_16end_L1D_112_MACHI_stabelized_aa.h5'
hUH_model_path=(os.path.join(DEH_DIR, hUH_model_file))

## LOAD FILES

### HYPSO 1

In [ ]:
L1A_array = [
    'yucatan2_2025-02-06T16-01-18Z-l1a_flat_L1A_120.npy',
    'kemigawa_2024-12-17T01-01-32Z-l1a_flat_L1A_120.npy', 
    'chapala_2025-02-24T16-52-47Z-l1a_flat_L1A_120.npy',
    'grizzlybay_2025-01-27T18-19-56Z-l1a_flat_L1A_120.npy',
    'victoriaLand_2025-02-07T20-35-33Z-l1a_flat_L1A_120.npy',
    'catala_2025-01-28T19-17-32Z-l1a_flat_L1A_120.npy',
    'khnifiss_2025-02-12T11-05-35Z-l1a_flat_L1A_120.npy',
    'menindee_2025-02-18T00-10-42Z-l1a_flat_L1A_120.npy',
    'tampa_2024-11-12T15-31-55Z-l1a_flat_L1A_120.npy',
    'falklandsatlantic_2024-12-18T13-25-18Z-l1a_flat_L1A_120.npy'
]


In [ ]:
HUH_labels=[
    'yucatan2_2025-02-06T16-01-18Z-l1a_flat_L1D_112_MACHI_16end_labels.npy',
    'kemigawa_2024-12-17T01-01-32Z-l1a_flat_L1D_112_MACHI_16end_labels.npy',
    'chapala_2025-02-24T16-52-47Z-l1a_flat_L1D_112_MACHI_16end_labels.npy',
    'grizzlybay_2025-01-27T18-19-56Z-l1a_flat_L1D_112_MACHI_16end_labels.npy',
    'victoriaLand_2025-02-07T20-35-33Z-l1a_flat_L1D_112_MACHI_16end_labels.npy',
    'catala_2025-01-28T19-17-32Z-l1a_flat_L1D_112_MACHI_16end_labels.npy',
    'khnifiss_2025-02-12T11-05-35Z-l1a_flat_L1D_112_MACHI_16end_labels.npy',
    'menindee_2025-02-18T00-10-42Z-l1a_flat_L1D_112_MACHI_16end_labels.npy',
    'tampa_2024-11-12T15-31-55Z-l1a_flat_L1D_112_MACHI_16end_labels.npy',
    'falklandsatlantic_2024-12-18T13-25-18Z-l1a_flat_L1D_112_MACHI_16end_labels.npy'
]

In [ ]:
SVM_predictions = [
    'yucatan2_2025_02_06_pred.npy',
    'kemigawa_2024_12_17_pred.npy',
    'chapala_2025_02_24_pred.npy',
    'grizzlybay_2025_01_27_pred.npy',
    'victoriaLand_2025_02_07_pred.npy',
    'catala_2025_01_28_pred.npy',
    'khnifiss_2025_02_12_pred.npy',
    'menindee_2025_02_18_pred.npy',
    'tampa_2024_11_12_pred.npy',
    'falklandsatlantic_2024_12_18_pred.npy'
]

In [ ]:
binary_predictions = [
    'caspiansea1_2025-04-08T07-11-56Z_16end.bin',
    'tampa_2024-11-12T15-31-55Z_16end.bin',
    'vancouver_2025-05-04T19-12-24Z_16end.bin',
    'grizzlybay_2025-05-30T18-20-50Z_16end.bin'
]
decoded_predictions = [
    'caspiansea1_2025-04-08T07-11-56Z_16end.npy',
    'tampa_2024-11-12T15-31-55Z_16end.npy',
    'vancouver_2025-05-04T19-12-24Z_16end.npy',
    'grizzlybay_2025-05-30T18-20-50Z_16end.npy'
]


In [ ]:
image_file          ='vancouver_2025-05-04T19-12-24Z-l1a_flat_L1D_112_MACHI.npy'
raw_image_file      ='vancouver_2025-05-04T19-12-24Z-l1a_flat_L1A_120.npy'
HUH_label_file      ='vancouver_2025-05-04T19-12-24Z-l1a_flat_L1D_112_MACHI_16end_labels.npy'
SVM_pred_file       ='vancouver_2025_05_04_L1A_120_pred.npy'
DECODED_pred_file   ="vancouver_2025-05-04T19-12-24Z_16end.npy"

In [ ]:
image_file          ='caspiansea1_2025-04-08T07-11-56Z-l1a_flat_L1D_112_MACHI.npy'
raw_image_file      ='caspiansea1_2025-04-08T07-11-56Z-l1a_flat_L1A_120.npy'
HUH_label_file      ='caspiansea1_2025-04-08T07-11-56Z-l1a_flat_L1D_112_MACHI_16end_labels.npy'
SVM_pred_file       ='caspiansea1_2025_04_08_pred.npy'
DECODED_pred_file   ="caspiansea1_2025-04-08T07-11-56Z_16end.npy"

In [ ]:
image_file          ='tampa_2024-11-12T15-31-55Z-l1a_flat_L1D_112_MACHI.npy'
raw_image_file      ='tampa_2024-11-12T15-31-55Z-l1a_flat_L1A_120.npy'
HUH_label_file      ='tampa_2024-11-12T15-31-55Z-l1a_flat_L1D_112_MACHI_16end_labels.npy'
SVM_pred_file       ='tampa_2024_11_12_pred.npy'
DECODED_pred_file   ="tampa_2024-11-12T15-31-55Z_16end.npy"

In [ ]:
image_path          =os.path.join(IMAGES_DIR, image_file)
raw_image_path      =os.path.join(IMAGES_DIR, raw_image_file)
HUH_label_path      =os.path.join(LABELS_DIR, HUH_label_file)
SVM_pred_path       =os.path.join(PRED_DIR, SVM_pred_file)
DECODED_pred_path   =os.path.join(DECODED_DIR, DECODED_pred_file)

### HYPSO 2

In [ ]:
H2_10_label_file='H2_10_L1D_112_MACHI_16end_labels.npy'
H2_10_label_path=os.path.join(LABELS_DIR,H2_10_label_file)

H2_10_L1A_120                 = np.load(os.path.join(IMAGES_DIR,'H2_10_L1A_120.npy'))
H2_10_L1D_112_MACHI           = np.load(os.path.join(IMAGES_DIR,'H2_10_L1D_112_MACHI.npy'))
H2_10_L1D_112_MACHI_labels    = np.load(os.path.join(LABELS_DIR,'H2_10_L1D_112_MACHI_16end_labels.npy'), allow_pickle=True).item()

In [ ]:
H2_L1D_files = [
    'yucatan1_2025-04-01_flat_L1D_112_H2_MACHI.npy',
    'kemigawa_2025-01-22_flat_L1D_112_H2_MACHI.npy',
    'chapala_2025-03-25_flat_L1D_112_H2_MACHI.npy',
    'grizzlybay_2025-01-22_flat_L1D_112_H2_MACHI.npy',
    'victoriaLand_2025-03-16_flat_L1D_112_H2_MACHI.npy',
    'mjosa_2025-05-12_flat_L1D_112_H2_MACHI.npy',
    'gobabeb_2025-04-25_flat_L1D_112_H2_MACHI.npy',
    'menindee_2025-05-09_flat_L1D_112_H2_MACHI.npy',
    'erie_2025-05-10_flat_L1D_112_H2_MACHI.npy',
    'falklandsatlantic_2025-03-03_flat_L1D_112_H2_MACHI.npy'
]

In [ ]:
H2_L1A_files = [
    'yucatan1_2025-04-01_flat_L1A_120_H2.npy',
    'kemigawa_2025-01-22_flat_L1A_120_H2.npy',
    'chapala_2025-03-25_flat_L1A_120_H2.npy',
    'grizzlybay_2025-01-22_flat_L1A_120_H2.npy',
    'victoriaLand_2025-03-16_flat_L1A_120_H2.npy',
    'mjosa_2025-05-12_flat_L1A_120_H2.npy',
    'gobabeb_2025-04-25_flat_L1A_120_H2.npy',
    'menindee_2025-05-09_flat_L1A_120_H2.npy',
    'erie_2025-05-10_flat_L1A_120_H2.npy',
    'falklandsatlantic_2025-03-03_flat_L1A_120_H2.npy'
]

In [ ]:
H2_HUH_labels=[
    'yucatan1_2025-04-01_flat_L1D_112_H2_MACHI_16end_labels.npy',
    'kemigawa_2025-01-22_flat_L1D_112_H2_MACHI_16end_labels.npy',
    'chapala_2025-03-25_flat_L1D_112_H2_MACHI_16end_labels.npy',
    'grizzlybay_2025-01-22_flat_L1D_112_H2_MACHI_16end_labels.npy',
    'victoriaLand_2025-03-16_flat_L1D_112_H2_MACHI_16end_labels.npy',
    'mjosa_2025-05-12_flat_L1D_112_H2_MACHI_16end_labels.npy',
    'gobabeb_2025-04-25_flat_L1D_112_H2_MACHI_16end_labels.npy',
    'menindee_2025-05-09_flat_L1D_112_H2_MACHI_16end_labels.npy',
    'erie_2025-05-10_flat_L1D_112_H2_MACHI_16end_labels.npy',
    'falklandsatlantic_2025-03-03_flat_L1D_112_H2_MACHI_16end_labels.npy'
]

In [ ]:
H2_SVM_predictions=[
    'yucatan1_2025-04-01_flat_L1A_120_H2_16end_pred.npy',
    'kemigawa_2025-01-22_flat_L1A_120_H2_16end_pred.npy',
    'chapala_2025-03-25_flat_L1A_120_H2_16end_pred.npy',
    'grizzlybay_2025-01-22_flat_L1A_120_H2_16end_pred.npy',
    'victoriaLand_2025-03-16_flat_L1A_120_H2_16end_pred.npy',
    'mjosa_2025-05-12_flat_L1A_120_H2_16end_pred.npy',
    'gobabeb_2025-04-25_flat_L1A_120_H2_16end_pred.npy',
    'menindee_2025-05-09_flat_L1A_120_H2_16end_pred.npy',
    'erie_2025-05-10_flat_L1A_120_H2_16end_pred.npy',
    'falklandsatlantic_2025-03-03_flat_L1A_120_H2_16end_pred.npy'
]

## PLOTTING FUNCTIONS 

In [ ]:
def create_discrete_colormap(n_classes=16, reverse=False):
    """
    Create a discrete colormap with fixed colors for each class.
    """
    # Get colors from viridis
    viridis = plt.cm.viridis
    # Create a fixed set of n_classes colors
    if reverse:
        colors = [viridis(1 - i/(n_classes-1)) for i in range(n_classes)]
    else:
        colors = [viridis(i/(n_classes-1)) for i in range(n_classes)]
    # # Add transparent color for 0 (background)
    # colors.insert(0, (0,0,0,0))
    # Create discrete colormap
    return plt.matplotlib.colors.ListedColormap(colors)

def create_custom_colormap():
    """
    Create a custom colormap with manually selected colors for 16 classes.
    Colors are specified in RGB format (values between 0 and 1).
    """

    colors = [
        (0, 0.2, 0),          
        (0, 0.25, 0),          
        (0, 0.30, 0),           
        (0, 0.35, 0),           
        (0.80, 0.75, 0.57),           
        (0, 0.2, 0.2),           
        (0.75, 0.75, 0.75),          
        (0.9, 0.9, 0.9),       
        (0, 0, 0.3),       
        (0, 0, 0.35),      
        (0, 0, 0.4),      
        (0, 0, 0.45),    
        (0, 0, 0.5),      
        (0, 0, 0.55),     
        (0, 0, 0.6),
        (1, 1, 1)        
    ]
    
    # Create discrete colormap
    return plt.matplotlib.colors.ListedColormap(colors)

realistic_cmap = create_custom_colormap()

discrete_cmap=create_discrete_colormap(16)

custom_cmap = realistic_cmap

In [ ]:
def load_path(path):
    """
    Load a dictionary or array from a .npy file
    
    Parameters:
    -----------
    dict_path : str
        Path to the .npy file containing the dictionary or array
        
    Returns:
    --------
    dict or array
        Dictionary or array loaded from the .npy file
    """
    loaded_dict = np.load(path, allow_pickle=True)
    if isinstance(loaded_dict, np.ndarray):
        if loaded_dict.size == 1:
            return loaded_dict.item()
        return loaded_dict

def extract_rgb(image_path, bands=(69, 46, 26)):
    """extract normalized RGB composite from hyperspectral data 
    
    Args:
        image_path: Path to hyperspectral data cube in flat format(h*w, n_bands)
        bands: Tuple of band indices to use for R,G,B channels (default: 69,46,26)

    Returns:
        rgb_normalized: Normalized RGB composite image
    """
    # Load and reshape flat data into cube format
    data = np.load(image_path)
    rgb_cube = data.reshape(-1, HYPSO_HEIGHT, data.shape[-1])
    
    # Extract RGB bands and stack them
    rgb = np.stack([rgb_cube[..., b] for b in bands], axis=-1)
    
    # Normalize RGB values to [0,1] range
    rgb_normalized = (rgb - np.min(rgb)) / (np.max(rgb) - np.min(rgb))
    # Rotate and flip for proper orientation
    rgb_normalized = np.rot90(rgb_normalized)
    rgb_normalized = np.flipud(rgb_normalized)

    return rgb_normalized
    
def plot_rgb(image_path, bands=(69, 46, 26), figsize=(15, 10)):
    """Plot RGB composite from hyperspectral data.
    
    Args:
        image_path: Path to hyperspectral data
        bands: RGB band indices (default: 69,46,26)
        figsize: Figure size(default: 15x10)
    """
    rgb_normalized = extract_rgb(image_path, bands)
    plt.figure(figsize=figsize)
    plt.imshow(rgb_normalized, aspect=0.1)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def plot_HUH_label(labels_filepath, key=None, figsize=(15, 10), cmap='viridis'):
    
    labels = load_path(labels_filepath)
    plt.figure(figsize=figsize)
    
    if key == None:
        # Get max level nodes
        max_level = max(len(k) for k in labels.keys())
        max_level_keys = [k for k in labels.keys() if len(k) == max_level]
        max_level_keys.sort(key=lambda x: int(x, 2))
        
        combined_label = np.zeros((HYPSO_WIDTH, HYPSO_HEIGHT), dtype=int)
        for i, k in enumerate(max_level_keys):
            try:
                pred_reshaped = labels[k].reshape(HYPSO_WIDTH, HYPSO_HEIGHT)
                combined_label[pred_reshaped == True] = i + 1
            except Exception as e:
                print(f"Error reshaping key '{k}': {e}")
                continue
        plt.imshow(np.flipud(np.rot90(combined_label)), cmap=cmap, vmin=0, aspect=0.1)
    else:
        if key not in labels:
            print(f"Key '{key}' not found in labelss")
            return
        pred_data = labels[key].reshape(HYPSO_WIDTH, HYPSO_HEIGHT)
        plt.imshow(np.flipud(np.rot90(pred_data)), cmap=cmap, vmin=0, vmax=1, aspect=0.1)
    
    
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def plot_HUH_labels_for_level(labels_filepath, level=None, cmap=None, figsize=(15,10)):
    """
    Visualizes binary labels for hierarchical nodes at a specified level.
    Highlights overlapping regions between different node labels.
    
    Args:
        labels_filepath (str): Full path to the .npy file containing binary labels
        level (int, optional): Hierarchical level to plot. If None, uses maximum level.
    """
    # Load and prepare data
    labels = load_path(labels_filepath)
    max_level = max(len(k) for k in labels.keys())
    level = max_level if level is None or level > max_level else level
    
    # Get nodes at specified level
    level_keys = [k for k in labels.keys() if len(k) == level]
    if not level_keys:
        print(f"No nodes found at level {level}")
        return

    # Setup visualization
    plt.figure(figsize=figsize)
    combined_label = np.zeros((HYPSO_WIDTH, HYPSO_HEIGHT), dtype=int)
    
    # Map each node to a unique label value
    for i, k in enumerate(level_keys):
        try:
            pred_reshaped = labels[k].reshape(HYPSO_WIDTH, HYPSO_HEIGHT)
            combined_label[pred_reshaped == True] = i + 1
        except Exception as e:
            print(f"Error reshaping key '{k}': {e}")
            continue
    
    # Create a colormap scaled to the number of nodes
    num_nodes = len(level_keys)

    if cmap is None:
        scaled_cmap = create_discrete_colormap(num_nodes-1) 
    else:
        scaled_cmap = cmap
    
    # Plot the combined labels
    plt.imshow(np.flipud(np.rot90(combined_label)), cmap=scaled_cmap, vmin=0, aspect=0.1)
    
    # Configure plot
    plt.title(f'Binary Labels for Level {level} Nodes')
    legend_elements = [plt.Rectangle((0,0),1,1, facecolor=scaled_cmap((i+1)/num_nodes)) for i in range(num_nodes)]
    legend_labels = list(level_keys)
    plt.legend(legend_elements, legend_labels, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def plot_HUH_unbinarized(deh_obj, node=None, cmap=None, figsize=(15,10)):
   
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax.imshow(np.flipud(np.rot90(deh_obj.nodes[node].map.reshape(deh_obj.plot_size))),aspect=0.1,  cmap=cmap)
    # ax.imshow(np.rot90(deh_obj.nodes[node].map.reshape(deh_obj.plot_size)), aspect=0.1, cmap=cmap)
    # ax.set_ylabel(deh_obj.nodes[node_key].map.astype(np.float32).sum())
    ax.set_title(node)
    fig.savefig(f'node_{node}.png', bbox_inches='tight', dpi=300)
    plt.show()

def plot_all_HUH_unbinarized(image_filepath, cmap=None, figsize=(15,10)):
    deh_obj = deh.DEH(no_negative_residuals=True)
    deh_obj.load(hUH_model_path)
    deh_obj.plot_size=(HYPSO_WIDTH,HYPSO_HEIGHT)
    image=load_path(image_filepath)
    pred=deh_obj.simple_predict(image)
    node_keys = sorted(deh_obj.nodes.keys())
    for node_key in node_keys:
        # print(f"Plotting node: {node_key}")
        plot_HUH_unbinarized(deh_obj, node_key, cmap=cmap, figsize=figsize)

def plot_spectra_for_level(save_file_path, image_path, level_input=None):
    """
    Plot spectral signatures for nodes at a specified level from a DEH model.
    (Using modified deh.py for altered DEH_temp.display_spectra(level_nodes))
    Args:
        save_file_path: Path to saved DEH model
        image_path: Path to flattened input data cube
        level_input: Hierarchical level to display (defaults to deepest level)
    """
    DEH_temp = deh.DEH(no_negative_residuals=True)
    DEH_temp.load(save_file_path)
    cm_input = np.load(image_path)
    cube_image = cm_input.reshape(-1,HYPSO_HEIGHT,cm_input.shape[-1])

    DEH_temp.plot_size = (cube_image.shape[0],cube_image.shape[1])
    DEH_temp.simple_predict(cm_input)
    nodes = DEH_temp.nodes
    num_levels = max(len(node) for node in DEH_temp.nodes)
    
    # If level_input is empty or invalid, use the last level
    if level_input == None or not isinstance(level_input, int) or level_input > num_levels or level_input < 1:
        level = num_levels
    else:
        level = level_input
        
    # Get nodes at specified level
    level_nodes = [node for node in nodes if len(node) == level]
    print("using level", level, "with", len(level_nodes), "nodes")
    DEH_temp.display_spectra(level_nodes, wl=wavelengths)

def plot_spectra_for_nodes(save_file_path, image_path, node_keys, ):
    """
    Plot spectral signatures for specified nodes from a DEH model.
    (Using modified deh.py for altered DEH_temp.display_spectra(node_keys))
    Args:
        save_file_path: Path to saved DEH model
        image_path: Path to flattened input data cube
        node_keys: List of node keys to display spectra for
    """
    DEH_temp = deh.DEH(no_negative_residuals=True)
    DEH_temp.load(save_file_path)
    cm_input = np.load(image_path)
    cube_image = cm_input.reshape(-1,HYPSO_HEIGHT,cm_input.shape[-1])

    DEH_temp.plot_size = (cube_image.shape[0],cube_image.shape[1])
    DEH_temp.simple_predict(cm_input)
    
    print(f"Plotting spectra for {len(node_keys)} nodes")
    DEH_temp.display_spectra(node_keys, wl=wavelengths)

def plot_overlay(rgb_image_path, label_path, node_key, alpha=0.5, figsize=(15, 10), rgb_bands=(69, 46, 26)):
    """
    Create an overlay visualization of a hierarchical label node on an RGB image.
    
    This function generates two side-by-side plots:
    1. The original RGB image with the specified node highlighted in red
    2. The original RGB image with the inverse of the node highlighted in red
    
    The overlay uses semi-transparent red coloring (controlled by alpha parameter)
    to show where the node occurs in the image.
    
    Parameters
    ----------
    rgb_image_path : str
        Path to the RGB image file (.npy)
    label_path : str
        Path to the hierarchical label file (.npy)
    node_key : str
        Key identifying the specific node to visualize
    alpha : float, default=0.5
        Transparency level of the overlay (0-1)
    figsize : tuple, default=(15, 10)
        Figure size in inches (width, height)
    rgb_bands : tuple, default=(69, 46, 26)
        Indices of bands to use for RGB visualization
    """
    # Load and prepare RGB image
    rgb_normalized = extract_rgb(rgb_image_path, rgb_bands)
    
    # Load label data
    labels = load_path(label_path)
    
    if node_key not in labels:
        return
    
    # Get raw mask and reshape to match image dimensions
    raw_mask = labels[node_key].astype(int)
    
    # Reshape to match the dimensions expected by HYPSO
    mask_reshaped = raw_mask.reshape(HYPSO_WIDTH, HYPSO_HEIGHT)
    
    # Rotate and flip to match the RGB image orientation
    mask_display = np.rot90(mask_reshaped)
    mask_display = np.flipud(mask_display)
    
    # Create inverse mask directly 
    inverse_mask = np.ones_like(mask_display) - mask_display
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Create overlay masks
    red_mask = np.zeros(mask_display.shape + (4,))  # RGBA
    red_mask[mask_display > 0.5] = [1, 0, 0, alpha]  # Red with alpha
    
    inverse_red_mask = np.zeros(inverse_mask.shape + (4,))  # RGBA
    inverse_red_mask[inverse_mask > 0.5] = [1, 0, 0, alpha]  # Red with alpha
    
    # Plot directly without using colormap
    ax1.imshow(rgb_normalized, aspect=0.1)
    ax1.imshow(red_mask, aspect=0.1)
    ax1.set_title(f'Node {node_key} Overlay')
    ax1.axis('off')
    
    ax2.imshow(rgb_normalized, aspect=0.1)
    ax2.imshow(inverse_red_mask, aspect=0.1)
    ax2.set_title(f'Inverse Node {node_key} Overlay')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

def plot_HUH_overlay_for_level(image_path, HUH_label_path, level=None, alpha=0.5, figsize=(15, 10), rgb_bands=(69, 46, 26)):
    """
    Plot overlay for all nodes in a specific level by calling Plot_HUH_overlay for each node.
    
    Parameters:
    -----------
    image_path : str
        Path to the RGB image file
    HUH_label_path : str
        Path to the label file
    level : int, optional
        Level number to plot nodes from. If None or invalid, uses the maximum level (default: None)
    alpha : float, optional
        Transparency of the overlay (default: 0.5)
    figsize : tuple, optional
        Size of the figure (width, height) in inches (default: (10, 10))
    rgb_bands : tuple, optional
        RGB band indices to use (default: (69, 46, 26))
    """
    # Load the label data
    labels = np.load(HUH_label_path, allow_pickle=True).item()
    
    # Get all possible levels
    available_levels = set(len(key) for key in labels.keys())
    
    # If level is None or not in available levels, use maximum level
    if level is None or level not in available_levels:
        level = max(available_levels)
        print(f"Using maximum level: {level}")
    
    # Get all nodes for the specified level
    level_nodes = [key for key in labels.keys() if len(key) == level]
    
    if not level_nodes:
        print(f"No nodes found for level {level}")
        return
    
    # Plot each node using Plot_HUH_overlay
    for node_key in level_nodes:
        plot_overlay(image_path, HUH_label_path, node_key, alpha, figsize, rgb_bands)

def plot_prediction(prediction_path, key=None, figsize=(15, 10), cmap='viridis'):
    """
    Plot prediction results from a prediction file.
    
    Parameters:
    -----------
    prediction_path : str
        Path to the prediction file to load and plot
    key : str, optional
        Specific key to plot from the prediction dictionary. If empty string (default),
        combines all predictions into a single plot
    figsize : tuple, optional
        Figure size in inches (width, height), default is (15, 10)
    cmap : str, optional
        Colormap to use for visualization, default is 'viridis'
        
    Returns:
    --------
    None
        Displays the plot using matplotlib
    """
    
    prediction = load_path(prediction_path)
    plt.figure(figsize=figsize)
    
    if key == None:
        combined_pred = np.zeros((HYPSO_WIDTH, HYPSO_HEIGHT), dtype=int)
        for i, k in enumerate(prediction.keys()):
            if k == '':
                continue
            try:
                pred_reshaped = prediction[k].reshape(HYPSO_WIDTH, HYPSO_HEIGHT)
                combined_pred[pred_reshaped == True] = i + 1
            except Exception as e:
                print(f"Error reshaping key '{k}': {e}")
                continue
        plt.imshow(np.flipud(np.rot90(combined_pred)), cmap=cmap, vmin=0, vmax=15, aspect=0.1)
    else:
        if key not in prediction:
            print(f"Key '{key}' not found in predictions")
            return
        pred_data = prediction[key].reshape(HYPSO_WIDTH, HYPSO_HEIGHT)
        plt.imshow(np.flipud(np.rot90(pred_data)), cmap=cmap, vmin=0, vmax=1, aspect=0.1)
    
    
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def plot_decoded_prediction(decoded_pred_path, figsize=(15, 10), cmap='viridis'):
    """
    Plot decoded prediction with consistent color mapping.
    
    Parameters:
    -----------
    decoded_pred_path : str
        Path to the decoded prediction file
    figsize : tuple
        Figure size (width, height)
    cmap : str or matplotlib.colors.Colormap
        Colormap to use
    add_one : bool
        Whether to add 1 to all class values
    """
    image = np.load(decoded_pred_path)
    image = np.flipud(np.rot90(image))
    
    # Add 1 to all class values to retain the background value(0) for no class
    image = image + 1
    
    plt.figure(figsize=figsize)
    
    # Use proper normalization for discrete colormaps
    if isinstance(cmap, plt.matplotlib.colors.ListedColormap):
        # For discrete colormaps, we need to be explicit about boundaries
        plt.imshow(image, aspect=0.1, cmap=cmap, vmin=0)
    else:
        # For continuous colormaps like 'viridis'
        plt.imshow(image, aspect=0.1, cmap=cmap)
        
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def plot_decoded_overlay(rgb_image_path, label_path, node_key, alpha=0.5, figsize=(15, 10), rgb_bands=(69, 46, 26)):
    """
    Create an overlay visualization of a hierarchical label node on an RGB image.
    
    This function generates two side-by-side plots:
    1. The original RGB image with the specified node highlighted in red
    2. The original RGB image with the inverse of the node highlighted in red
    
    The overlay uses semi-transparent red coloring (controlled by alpha parameter)
    to show where the node occurs in the image.
    
    Parameters
    ----------
    rgb_image_path : str
        Path to the RGB image file (.npy)
    label_path : str
        Path to the hierarchical label file (.npy)
    node_key : str
        Key identifying the specific node to visualize
    alpha : float, default=0.5
        Transparency level of the overlay (0-1)
    figsize : tuple, default=(15, 10)
        Figure size in inches (width, height)
    rgb_bands : tuple, default=(69, 46, 26)
        Indices of bands to use for RGB visualization
    """
    # Load and prepare RGB image
    rgb_normalized = extract_rgb(rgb_image_path, rgb_bands)
    
    # Load label data
    labels = load_path(label_path)
    
    # Create binary mask where values match node_key
    raw_mask = np.zeros_like(labels, dtype=int)
    raw_mask[labels == node_key] = 1

    # Reshape to match the dimensions expected by HYPSO
    mask_reshaped = raw_mask.reshape(HYPSO_WIDTH, HYPSO_HEIGHT)
    
    # Rotate and flip to match the RGB image orientation
    mask_display = np.rot90(mask_reshaped)
    mask_display = np.flipud(mask_display)
    
    # Create inverse mask directly 
    inverse_mask = np.ones_like(mask_display) - mask_display
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Create overlay masks
    red_mask = np.zeros(mask_display.shape + (4,))  # RGBA
    red_mask[mask_display > 0.5] = [1, 0, 0, alpha]  # Red with alpha
    
    inverse_red_mask = np.zeros(inverse_mask.shape + (4,))  # RGBA
    inverse_red_mask[inverse_mask > 0.5] = [1, 0, 0, alpha]  # Red with alpha
    
    # Plot directly without using colormap
    ax1.imshow(rgb_normalized, aspect=0.1)
    ax1.imshow(red_mask, aspect=0.1)
    ax1.set_title(f'Node {node_key} Overlay')
    ax1.axis('off')
    
    ax2.imshow(rgb_normalized, aspect=0.1)
    ax2.imshow(inverse_red_mask, aspect=0.1)
    ax2.set_title(f'Inverse Node {node_key} Overlay')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

def plot_overlay_comparison(rgb_image_path, label_path1, label_path2, node_key, alpha=0.5, figsize=(15, 10), rgb_bands=(69, 46, 26), invert=False):
    """
    Create an overlay visualization comparing two hierarchical label nodes on an RGB image.
    
    This function generates two side-by-side plots:
    1. The original RGB image with the specified node from first label highlighted in red
    2. The original RGB image with the specified node from second label highlighted in red
    
    The overlay uses semi-transparent red coloring (controlled by alpha parameter)
    to show where the nodes occur in the image. The RGB image is normalized and displayed
    with the specified band indices for visualization.
    
    Parameters
    ----------
    rgb_image_path : str
        Path to the RGB image file (.npy)
    label_path1 : str
        Path to the first hierarchical label file (.npy)
    label_path2 : str
        Path to the second hierarchical label file (.npy)
    node_key : str
        Key identifying the specific node to visualize
    alpha : float, default=0.5
        Transparency level of the overlay (0-1)
    figsize : tuple, default=(15, 10)
        Figure size in inches (width, height)
    rgb_bands : tuple, default=(69, 46, 26)
        Indices of bands to use for RGB visualization (R,G,B)
    invert : bool, default=False
        Whether to invert the masks before displaying
    """
    # Load and prepare RGB image
    rgb_normalized = extract_rgb(rgb_image_path, rgb_bands)
    
    # Load label data
    labels1 = load_path(label_path1)
    labels2 = load_path(label_path2)
    
    if node_key not in labels1 or node_key not in labels2:
        return
    
    # Get raw masks and reshape to match image dimensions
    raw_mask1 = labels1[node_key].astype(int)
    raw_mask2 = labels2[node_key].astype(int)
    
    # Reshape to match the dimensions expected by HYPSO
    mask_reshaped1 = raw_mask1.reshape(HYPSO_WIDTH, HYPSO_HEIGHT)
    mask_reshaped2 = raw_mask2.reshape(HYPSO_WIDTH, HYPSO_HEIGHT)
    
    # Rotate and flip to match the RGB image orientation
    mask_display1 = np.rot90(mask_reshaped1)
    mask_display1 = np.flipud(mask_display1)
    
    mask_display2 = np.rot90(mask_reshaped2)
    mask_display2 = np.flipud(mask_display2)
    
    # Invert masks if requested
    if invert:
        mask_display1 = 1 - mask_display1
        mask_display2 = 1 - mask_display2
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Create overlay masks
    red_mask1 = np.zeros(mask_display1.shape + (4,))  # RGBA
    red_mask1[mask_display1 > 0.5] = [1, 0, 0, alpha]  # Red with alpha
    
    red_mask2 = np.zeros(mask_display2.shape + (4,))  # RGBA
    red_mask2[mask_display2 > 0.5] = [1, 0, 0, alpha]  # Red with alpha
    
    # Plot directly without using colormap
    ax1.imshow(rgb_normalized, aspect=0.1)
    ax1.imshow(red_mask1, aspect=0.1)
    if invert:
        ax1.set_title(f'Node {node_key} Overlay (Prediction 1) Inverted')
    else:
        ax1.set_title(f'Node {node_key} Overlay (Prediction 1)')
    ax1.axis('off')
    
    ax2.imshow(rgb_normalized, aspect=0.1)
    ax2.imshow(red_mask2, aspect=0.1)
    if invert:
        ax2.set_title(f'Node {node_key} Overlay (Prediction 2) Inverted')
    else:
        ax2.set_title(f'Node {node_key} Overlay (Prediction 2)')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

def plot_overlay_comparison_for_level(image_path, label_path1, label_path2, level=None, alpha=0.5, figsize=(15, 10), rgb_bands=(69, 46, 26), invert=False):
    """
    Plot overlay for all nodes in a specific level by calling Plot_HUH_overlay for each node.
    
    Parameters:
    -----------
    image_path : str
        Path to the RGB image file
    HUH_label_path : str
        Path to the label file
    level : int, optional
        Level number to plot nodes from. If None or invalid, uses the maximum level (default: None)
    alpha : float, optional
        Transparency of the overlay (default: 0.5)
    figsize : tuple, optional
        Size of the figure (width, height) in inches (default: (10, 10))
    rgb_bands : tuple, optional
        RGB band indices to use (default: (69, 46, 26))
    """
    # Load the label data
    labels1 = np.load(label_path1, allow_pickle=True).item()
    labels2 = np.load(label_path2, allow_pickle=True).item()
    
    # Get all possible levels
    available_levels1 = set(len(key) for key in labels1.keys())
    available_levels2 = set(len(key) for key in labels2.keys())
    
    # If level is None or not in available levels, use maximum level
    if level is None or level not in available_levels1 or level not in available_levels2:
        level = max(available_levels1, available_levels2)
        print(f"Using maximum level: {level}")
    
    # Get all nodes for the specified level
    level_nodes1 = [key for key in labels1.keys() if len(key) == level]
    level_nodes2 = [key for key in labels2.keys() if len(key) == level]
    common_nodes = list(set(level_nodes1).intersection(set(level_nodes2)))
    # Sort common nodes by their binary value
    common_nodes.sort(key=lambda x: int(x, 2))
    
    if not level_nodes1 or not level_nodes2:
        print(f"No nodes found for level {level}")
        return
    
    # Plot each node using Plot_HUH_overlay
    for node_key in common_nodes:
            plot_overlay_comparison(image_path, label_path1, label_path2, node_key, alpha, figsize, rgb_bands, invert)

def plot_rgb_labels_and_predictions(image_array=None, label_array=None, prediction_array=None, cmap='viridis'):
    if image_array is not None:
        n_entries = len(image_array)
    elif label_array is not None:
        n_entries = len(label_array)
    elif prediction_array is not None:
        n_entries = len(prediction_array)
    else:
        print("plot_rgb_labels_and_predictions: No data provided")
        return
    for i in range(n_entries):
        if image_array is not None:
            image_path=os.path.join(IMAGES_DIR,image_array[i])
            plot_rgb(image_path)
        if label_array is not None:
            label_path=os.path.join(LABELS_DIR,label_array[i])
            plot_HUH_label(label_path, cmap=cmap)
        if prediction_array is not None:
            predicion_path=os.path.join(PRED_DIR,prediction_array[i])
            plot_prediction(predicion_path, cmap=cmap)    


## PLOT

### Plot RGB

In [ ]:
plot_rgb(raw_image_path)

### Plot labels for hUH label file(.npy) 

In [ ]:
# plot_HUH_label(HUH_label_path, cmap=custom_cmap)
plot_HUH_label(HUH_label_path, cmap='viridis')


In [ ]:
plot_HUH_labels_for_level(HUH_label_path, level=7)

### PLOT SVM TREE prediction(.npy)

In [ ]:
plot_prediction(SVM_pred_path, cmap=custom_cmap)

### Plot decoded binary SVM TREE prediction(.npy)

In [ ]:
plot_decoded_prediction(DECODED_pred_path,cmap=custom_cmap)

### Decode binary SVM TREE prediction(.bin)

In [ ]:
def decode_binary_file(binary_file, output_dir):
    
    # Check if file exists, try different extensions if not
    file_to_open = binary_file
    if not os.path.exists(file_to_open):
        # Try with .bin extension
        if os.path.exists(binary_file + ".bin"):
            file_to_open = binary_file + ".bin"
            #print(f"Found file with .bin extension: {file_to_open}")
        # Try with common extensions
        elif os.path.exists(binary_file + ".dat"):
            file_to_open = binary_file + ".dat"
            #print(f"Found file with .dat extension: {file_to_open}")
        else:
            # List similar files in the directory
            try:
                base_dir = os.path.dirname(binary_file)
                base_name = os.path.basename(binary_file)
                print(f"Looking for files similar to {base_name} in {base_dir}")
                similar_files = [f for f in os.listdir(base_dir) if base_name in f]
                if similar_files:
                    print(f"Found similar files: {similar_files}")
                    # Use the first match
                    file_to_open = os.path.join(base_dir, similar_files[0])
                    print(f"Using: {file_to_open}")
                else:
                    print(f"No similar files found in {base_dir}")
            except Exception as e:
                print(f"Error listing directory: {str(e)}")
    
    # Create output directory if it doesn't exist
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
        print(f"Created output directory: {output_dir}")
    
    # Read the entire file
    try:
        with open(file_to_open, 'rb') as f:
            data = f.read()
    except FileNotFoundError:
        print(f"Error: Could not find file {file_to_open}")
        print("Please check that the file exists and the path is correct.")
        raise
    
    print(f"Successfully opened file: {file_to_open}")
    
    # First 8 bytes: timestamps
    # Next 16+ bytes: class list (terminated by 255s)
    
    # Extract class list
    class_list = []
    for i in range(8, len(data)):
        if i >= 24:  # Stop reading classes after 16 bytes
            break
        if data[i] == 255:  # Stop at first 255
            break
        class_list.append(data[i])
    
    print(f"Classes found in header: {class_list}")
    
    # Determine bits per pixel based on number of classes
    num_classes = len(class_list)
    if num_classes <= 2:
        bits_per_pixel = 1  # 1 bit per pixel, 8 pixels per byte
    elif num_classes <= 4:
        bits_per_pixel = 2  # 2 bits per pixel, 4 pixels per byte
    elif num_classes <= 16:
        bits_per_pixel = 4  # 4 bits per pixel, 2 pixels per byte
    else:
        bits_per_pixel = 8  # 8 bits per pixel, 1 pixel per byte
    
    print(f"Using {bits_per_pixel} bits per pixel for {num_classes} classes")
    
    # Skip header (first 24 bytes)
    raw_data = np.frombuffer(data[24:], dtype=np.uint8)
    
    # Create array to hold decoded labels
    total_pixels = HYPSO_HEIGHT * HYPSO_WIDTH
    decoded_labels = np.zeros(total_pixels, dtype=np.uint8)
    
    # Decode based on bits per pixel
    if bits_per_pixel == 1:
        # 8 pixels per byte
        for i in range(min(len(raw_data), total_pixels // 8 + 1)):
            for bit in range(8):
                pixel_idx = i*8 + bit
                if pixel_idx < total_pixels:
                    class_idx = (raw_data[i] >> bit) & 0x01
                    if class_idx < len(class_list):
                        decoded_labels[pixel_idx] = class_list[class_idx]
    
    elif bits_per_pixel == 2:
        # 4 pixels per byte
        for i in range(min(len(raw_data), total_pixels // 4 + 1)):
            for j in range(4):
                pixel_idx = i*4 + j
                if pixel_idx < total_pixels:
                    class_idx = (raw_data[i] >> (j*2)) & 0x03
                    if class_idx < len(class_list):
                        decoded_labels[pixel_idx] = class_list[class_idx]
    
    elif bits_per_pixel == 4:
        # 2 pixels per byte
        for i in range(min(len(raw_data), total_pixels // 2 + 1)):
            # First pixel in high 4 bits
            if i*2 < total_pixels:
                class_idx = (raw_data[i] >> 4) & 0x0F
                if class_idx < len(class_list):
                    decoded_labels[i*2] = class_list[class_idx]
            
            # Second pixel in low 4 bits
            if i*2 + 1 < total_pixels:
                class_idx = raw_data[i] & 0x0F
                if class_idx < len(class_list):
                    decoded_labels[i*2 + 1] = class_list[class_idx]
    
    else:  # bits_per_pixel == 8
        # 1 pixel per byte
        for i in range(min(len(raw_data), total_pixels)):
            class_idx = raw_data[i]
            if class_idx < len(class_list):
                decoded_labels[i] = class_list[class_idx]
    
    # Reshape to 2D
    decoded_image = decoded_labels.reshape(HYPSO_WIDTH,HYPSO_HEIGHT)
    
    # Get unique classes in the decoded image
    unique_classes = np.unique(decoded_image)
    print(f"Classes in decoded image: {unique_classes}")
    
    # Save as numpy array
    save_path = os.path.join(output_dir, f"{os.path.basename(binary_file).replace('.bin', '.npy')}")
    np.save(save_path, decoded_image)
    print(f"Saved decoded labels to {save_path}")
    
    # Print class distribution
    for cls in unique_classes:
        count = np.sum(decoded_image == cls)
        percentage = 100.0 * count / total_pixels
        print(f"Class {cls}: {count} pixels ({percentage:.2f}%)")

def decode_binary_files_array(binary_files, output_dir):
    for binary_file in binary_files:
        binary_file_path = os.path.join(BINARY_PRED_DIR, binary_file)
        decode_binary_file(binary_file_path, output_dir=output_dir)

def plot_decoded_predictions(binary_files, cmap='viridis'):
    for binary_file in binary_files:
        binary_file = binary_file.replace('.bin', '.npy')
        binary_file_path = os.path.join(DECODED_DIR, binary_file)
        plot_decoded_prediction(binary_file_path, cmap=cmap)


In [ ]:
decode_binary_files_array(binary_predictions, output_dir=DECODED_DIR)

In [ ]:
plot_decoded_predictions(decoded_predictions, cmap=custom_cmap)

### Plot spectra for hUH model(.h5) and image (.npy)

In [ ]:
plot_spectra_for_level(hUH_model_path, image_path, wl=wavelengths)

In [ ]:
water_nodes=[
    '0010010',
    '0010011',
    '0010100',
    '0010101',
    '0010110',
    '0011000',
    '0100000'
]

land_nodes=[
    '0000000',
    '0000001',
    '0000010',
    '0000100'
]

mixed_nodes=[
    '0000110',
    '0001000',
    '0001100',
    '0010000',
]


In [ ]:
plot_spectra_for_nodes(hUH_model_path, image_path, node_keys=water_nodes)

In [ ]:
plot_spectra_for_nodes(hUH_model_path, image_path, node_keys=land_nodes)

In [ ]:
plot_spectra_for_nodes(hUH_model_path, image_path, node_keys=mixed_nodes)

In [ ]:
plot_spectra_for_nodes(hUH_model_path, image_path, node_keys=['1000000'])


### PLOT HUH_label(.npy) overlay on image(.npy)

In [ ]:
plot_overlay(raw_image_path, HUH_label_path, node_key='1')

In [ ]:
plot_HUH_overlay_for_level(raw_image_path, HUH_label_path, level=7)

## COMPARE PREDICTIONS AND GROUND TRUTH


In [ ]:
plot_rgb(raw_image_path)
plot_HUH_label(HUH_label_path, cmap=custom_cmap)
plot_prediction(SVM_pred_path, cmap=custom_cmap)
plot_decoded_prediction(DECODED_pred_path,cmap=custom_cmap)

In [ ]:
plot_rgb_labels_and_predictions(image_array=L1A_array, label_array=HUH_labels, prediction_array=SVM_predictions, cmap=custom_cmap)
plot_rgb_labels_and_predictions(image_array=H2_L1A_files, label_array=H2_HUH_labels, prediction_array=H2_SVM_predictions, cmap=custom_cmap)


In [ ]:
plot_overlay(raw_image_path, HUH_label_path, node_key='0000000')
plot_overlay(raw_image_path, SVM_pred_path, node_key='0000000')
plot_decoded_overlay(raw_image_path, DECODED_pred_path, node_key=0)

In [ ]:
plot_overlay_comparison(raw_image_path, HUH_label_path, SVM_pred_path, node_key='0000000')

In [ ]:
plot_overlay_comparison_for_level(raw_image_path, HUH_label_path, SVM_pred_path, level=7, invert=False)

# TEST

In [ ]:
colors = [
        (0, 0, 0.6),     
        (0, 0.35, 0),           
        (1, 1, 1)        
    ]
    
    # Create discrete colormap
cmap= plt.matplotlib.colors.ListedColormap(colors)

def compare_label_files(label_3class_path, label_16class_path, cmap='viridis'):
    """
    Compare two label files - one with 3 classes (.label) and one with 16 classes (.npy)
    Maps 16 classes to 3 classes before comparison
    
    Parameters:
    -----------
    label_3class_path : str
        Path to the 3-class label file (.label format)
    label_16class_path : str
        Path to the 16-class label file (.npy format)
        
    Returns:
    --------
    dict
        Dictionary containing comparison metrics
    """
    # Load the label files
    label_3class = np.fromfile(label_3class_path, dtype=np.uint8)
    label_16class = np.load(label_16class_path)
    
    # Reshape to HYPSO dimensions
    label_3class = label_3class.reshape(HYPSO_WIDTH,HYPSO_HEIGHT)
    label_3class = np.rot90(label_3class)
    label_16class = label_16class.reshape(HYPSO_WIDTH,HYPSO_HEIGHT)
    label_16class = np.rot90(label_16class)
    

    class_mapping = {
        0: 1,  
        1: 1,  
        2: 1,
        3: 1,
        4: 1,
        5: 1,
        6: 1,
        7: 0,
        8: 0,
        9: 0,
        10: 0,
        11: 0,
        12: 0,
        13: 0,
        14: 2,  
        15: 2
    }
    
    # Map 16-class labels to 3 classes
    mapped_16class = np.zeros_like(label_16class)
    for old_class, new_class in class_mapping.items():
        mapped_16class[label_16class == old_class] = new_class
    
    # Calculate accuracy using mapped labels
    total_pixels = label_3class.size
    correct_pixels = np.sum(label_3class == mapped_16class)
    accuracy = correct_pixels / total_pixels
    
    # Calculate per-class accuracy
    unique_classes = np.unique(label_3class)
    per_class_acc = {}
    for cls in unique_classes:
        class_mask = label_3class == cls
        if np.any(class_mask):
            class_acc = np.sum((label_3class == mapped_16class) & class_mask) / np.sum(class_mask)
            per_class_acc[cls] = class_acc
    
    # Create visualizations
    fig1 = plt.figure(figsize=(15, 5))
    ax1 = fig1.add_subplot(111)
    ax1.imshow(label_3class, cmap=cmap)
    ax1.set_title('1D-Justo-LiuNet sea land cloud')
    ax1.axis('off')
    ax1.set_aspect(0.1)
    
    fig2 = plt.figure(figsize=(15, 5))
    ax2 = fig2.add_subplot(111)
    ax2.imshow(mapped_16class, cmap=cmap)
    ax2.set_title('SVM BDT 16 classes mapped to sea land cloud')
    ax2.axis('off')
    ax2.set_aspect(0.1)
    
    plt.tight_layout()
    
    # Print metrics
    print(f"Overall Accuracy: {accuracy:.3f}")
    print("\nPer-class Accuracy:")
    for cls, acc in per_class_acc.items():
        print(f"Class {cls}: {acc:.3f}")
    
    return {
        'accuracy': accuracy,
        'per_class_accuracy': per_class_acc,
        'mapped_labels': mapped_16class
    }


In [ ]:
label_3class = r'data\decoded_predictions\tampa_2024-11-12T15-31-55Z_jon-cnn.labels'
label_16class= r'data\decoded_predictions\tampa_2024-11-12T15-31-55Z_16end.npy'
results = compare_label_files(label_3class, label_16class,cmap=cmap)
label_3class = r'data\decoded_predictions\vancouver_2025-05-04T19-12-24Z_sea-land-cloud.labels'
label_16class= r'data\decoded_predictions\vancouver_2025-05-04T19-12-24Z_16end.npy'
results = compare_label_files(label_3class, label_16class,cmap=cmap)

In [ ]:
plot_all_HUH_unbinarized(image_path, cmap='copper', figsize=(30,30))